In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import calendar
import tkinter as tk
from tkinter import filedialog, ttk
import datetime as dt


In [2]:
def generate_minute():
    return [f'{i:>02}' for i in range(0, 60)]


def generate_day():
    """Generate a list of date values for the next 30 days."""
    return [f'{i}' for i in range(1, 31)]


def generate_month():
    """Generate a list of date values for the next 30 days."""
    return list(calendar.month_name[1:])


def generate_year():
    """Generate a list of date values for the next 30 days."""
    return [f'{i}' for i in range(1990, 2032)]


def generate_hour():
    return [f'{i:>02}' for i in range(0, 24)]


class DateTimePickerRange(tk.Toplevel):
    start_date = dt.datetime.now()
    end_date = start_date + dt.timedelta(days=30)

    def __init__(self, parent, start_date, end_date):
        super().__init__(parent)
        self.result = None
        self.start_date = start_date
        self.end_date = end_date
        self.title("Date and Time Filter")
        self.geometry("540x240")
        self.resizable(False, False)

        # Main frame
        self.main_frame = tk.Frame(self)
        self.main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        start_box_frame = tk.LabelFrame(self.main_frame, text="Start Date & Time",
                                        bd=2, relief="solid")
        start_box_frame.pack(fill="x", pady=5)

        # Start date and time (aligned horizontally)
        start_frame = tk.Frame(start_box_frame)
        start_frame.pack(fill="x", pady=15)

        self.start_day_combo = ttk.Combobox(start_frame, values=generate_day(), state="readonly", width=3)
        self.start_day_combo.pack(side="left", padx=5)
        self.start_day_combo.set(self.start_date.date().day)

        self.start_month_combo = ttk.Combobox(start_frame, values=generate_month(), state="readonly", width=8)
        self.start_month_combo.pack(side="left", padx=5)
        self.start_month_combo.set(calendar.month_name[self.start_date.month])

        self.start_year_combo = ttk.Combobox(start_frame, values=generate_year(), state="readonly", width=4)
        self.start_year_combo.pack(side="left", padx=5)
        self.start_year_combo.set(self.start_date.year)

        tk.Frame(start_frame, width=30).pack(side="left")

        tk.Label(start_frame, text="Start Time:").pack(side="left", padx=5)

        self.start_hour_combo = ttk.Combobox(start_frame, values=generate_hour(), state="readonly", width=4)
        self.start_hour_combo.pack(side="left", padx=5)
        self.start_hour_combo.set(f'{self.start_date.hour:>02}')

        self.start_minute_combo = ttk.Combobox(start_frame, values=generate_minute(), state="readonly", width=4)
        self.start_minute_combo.pack(side="left", padx=5)
        self.start_minute_combo.set(f'{self.start_date.minute:>02}')

        end_box_frame = tk.LabelFrame(self.main_frame, text="End Date & Time",
                                      borderwidth=2, relief="solid",
                                      highlightbackground="lightgrey",
                                      highlightcolor="lightgrey")
        end_box_frame.pack(fill="x", pady=5)

        # Start date and time (aligned horizontally)
        end_frame = tk.Frame(end_box_frame)
        end_frame.pack(fill="x", pady=15)

        self.end_day_combo = ttk.Combobox(end_frame, values=generate_day(), state="readonly", width=3)
        self.end_day_combo.pack(side="left", padx=5)
        self.end_day_combo.set(self.end_date.date().day)

        self.end_month_combo = ttk.Combobox(end_frame, values=generate_month(), state="readonly", width=8)
        self.end_month_combo.pack(side="left", padx=5)
        self.end_month_combo.set(calendar.month_name[self.end_date.month])

        self.end_year_combo = ttk.Combobox(end_frame, values=generate_year(), state="readonly", width=4)
        self.end_year_combo.pack(side="left", padx=5)
        self.end_year_combo.set(self.end_date.year)

        tk.Frame(end_frame, width=30).pack(side="left")

        tk.Label(end_frame, text="End Time:").pack(side="left", padx=5)

        self.end_hour_combo = ttk.Combobox(end_frame, values=generate_hour(), state="readonly", width=4)
        self.end_hour_combo.pack(side="left", padx=5)
        self.end_hour_combo.set(f'{self.end_date.hour:>02}')

        self.end_minute_combo = ttk.Combobox(end_frame, values=generate_minute(), state="readonly", width=4)
        self.end_minute_combo.pack(side="left", padx=5)
        self.end_minute_combo.set(f'{self.end_date.minute:>02}')

        # Submit button (aligned to bottom-right)
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(fill="x", pady=10, anchor="se")

        submit_button = tk.Button(self.button_frame, text="Apply",
                                  command=self.on_ok)

        submit_button.pack(side="right")

        cancel_button = tk.Button(self.button_frame, text="Cancel", command=self.on_cancel)
        cancel_button.pack(side="right")

    def on_ok(self):
        s_day = self.start_day_combo.current() + 1
        s_month = self.start_month_combo.current() + 1
        s_year = self.start_year_combo.get()
        s_hour = self.start_hour_combo.current()
        s_minute = self.start_minute_combo.current()

        e_day = self.end_day_combo.current() + 1
        e_month = self.end_month_combo.current() + 1
        e_year = self.end_year_combo.get()
        e_hour = self.end_hour_combo.current()
        e_minute = self.end_minute_combo.current()

        self.start_date = dt.datetime(int(s_year), s_month, s_day, s_hour, s_minute, 0, 0)
        self.end_date = dt.datetime(int(e_year), e_month, e_day, e_hour, e_minute, 0, 0)

        self.result = (self.start_date, self.end_date)  # Capture the input
        self.destroy()  # Close the window

    def on_cancel(self):
        self.result = None  # No result (canceled)
        self.destroy()  # Close the window


In [3]:
_cell_width = 20
_cell_height = 30
_border_color = "dimgray"
_scrollbar_width = 16
_header_color = "grey"

In [4]:
class TableView:
    def __init__(self, root, header, data):

        self.container = tk.Canvas(root, highlightthickness=0)
        self.container.pack(fill="both", expand=True)

        self.header_canvas = tk.Canvas(self.container, highlightthickness=0)
        self.header_canvas.configure(height=_cell_height)
        self.header_canvas.pack(fill="x")

        self.body_canvas = tk.Canvas(self.container, highlightthickness=0)
        self.body_canvas.pack(fill="both", expand=True)

        # Scrollbars
        self.v_scroll = tk.Scrollbar(self.body_canvas, orient="vertical", command=self.body_canvas.yview)
        self.h_scroll = tk.Scrollbar(self.body_canvas, orient="horizontal", command=self.sync_scroll)

        self.body_canvas.configure(yscrollcommand=self.v_scroll.set, xscrollcommand=self.h_scroll.set)
        self.header_canvas.configure(xscrollcommand=self.h_scroll.set)

        self.v_scroll.pack(side="right", fill="y")
        self.h_scroll.pack(side="bottom", fill="x")

        # Create table
        total_width = self._create_header(header)
        total_height = self._create_table(header, data)
        self._set_scroll_padding(total_width, total_height)

        self._set_scroll_region()

    def _create_header(self, header):
        total = 0
        x1 = 0
        for header_text, cell_width in header:
            # Draw cell border
            x2 = x1 + (cell_width * _cell_width)  # Calculate the end x-coordinate
            y1, y2 = 0, _cell_height  # Fixed height for all headers

            self.header_canvas.create_rectangle(x1, y1, x2, y2,
                                                outline=_border_color, width=1, fill=_header_color)

            # Add cell text
            self.header_canvas.create_text(
                x1 + 10, y1 + _cell_height // 2,
                text=header_text, anchor="w",
                font=("Arial", 14, "bold"),
                # fill="black",
            )

            # Update x1 for the next cell
            x1 = x2
            total += (cell_width * _cell_width)

        # Adjust scroll region to fit content
        self.header_canvas.configure(scrollregion=self.header_canvas.bbox("all"))
        return total

    def _create_table(self, header, data):

        def on_enter(event):
            self.body_canvas.itemconfig(event.widget.find_withtag("current"), fill="lightblue", outline="black", width=3)

        def on_leave_even(event):
            self.body_canvas.itemconfig(event.widget.find_withtag("current"), fill="white", outline="black", width=1)

        def on_leave_odd(event):
            self.body_canvas.itemconfig(event.widget.find_withtag("current"), fill="gainsboro", outline="black", width=1)

        y1 = 0  # Starting y-coordinate (below the header)
        i = 0
        total = 0

        for row in data:
            x1 = 0  # Reset x1 for each row
            for cell, (_, cell_width) in zip(row, header):
                # Calculate cell boundaries
                x2 = x1 + (cell_width * _cell_width)
                y2 = y1 + _cell_height

                color = "white" if i % 2 == 0 else "gainsboro"
                tag = "h_even" if i % 2 == 0 else "h_odd"

                self.body_canvas.create_rectangle(x1, y1, x2, y2,
                                                  outline=_border_color,
                                                  width=1, fill=color, tags=tag)

                # Add cell text
                self.body_canvas.create_text(
                    x1 + 10, y1 + _cell_height // 2,
                    text=str(cell), anchor="w",
                    fill="black",
                    # tags=tag
                )

                # Update x1 for the next cell
                x1 = x2

            # Update y1 for the next row
            y1 += _cell_height
            i += 1

        # Adjust scroll region to fit content
        self.body_canvas.configure(scrollregion=self.body_canvas.bbox("all"))
        self.body_canvas.tag_bind("h_odd", "<Enter>", on_enter)  # Hover starts
        self.body_canvas.tag_bind("h_even", "<Enter>", on_enter)  # Hover starts
        self.body_canvas.tag_bind("h_odd", "<Leave>", on_leave_odd)
        self.body_canvas.tag_bind("h_even", "<Leave>", on_leave_even)

        return _cell_height * len(data)

    def _set_scroll_region(self):
        self.body_canvas.bind('<Configure>', self.update)
        self.bind_mouse_scroll(self.body_canvas, self.y_scroll)
        self.bind_mouse_scroll(self.h_scroll, self.x_scroll)
        self.bind_mouse_scroll(self.v_scroll, self.y_scroll)

        self.body_canvas.focus_set()

    def bind_mouse_scroll(self, parent, mode):
        # ~~ Windows only
        parent.bind("<MouseWheel>", mode)
        # ~~ Mac Horizontal Scroll
        parent.bind("<Shift-MouseWheel>", mode)
        # ~~ Unix only
        parent.bind("<Button-4>", mode)
        parent.bind("<Button-5>", mode)

    def y_scroll(self, event):
        if event.num == 5 or event.delta < 0:
            self.body_canvas.yview_scroll(1, "unit")
        elif event.num == 4 or event.delta > 0:
            self.body_canvas.yview_scroll(-1, "unit")

    def x_scroll(self, event):
        if event.num == 5 or event.delta < 0:
            self.body_canvas.xview_scroll(1, "unit")
            self.header_canvas.xview_scroll(1, "unit")
        elif event.num == 4 or event.delta > 0:
            self.body_canvas.xview_scroll(-1, "unit")
            self.header_canvas.xview_scroll(-1, "unit")

    def update(self, event):
        self.body_canvas.config(scrollregion=self.body_canvas.bbox('all'))
        self.header_canvas.config(scrollregion=self.header_canvas.bbox('all'))

    def sync_scroll(self, *args):
        """Sync scroll position for both canvases."""
        self.body_canvas.xview(*args)
        self.header_canvas.xview(*args)

    def _set_scroll_padding(self, width, height):

        x1 = width
        y1 = 0
        x2 = x1 + _scrollbar_width
        y2 = _cell_height

        self.header_canvas.create_rectangle(x1, y1, x2, y2, width=0)

        # add padding for scroll bars
        x1 = 0
        y1 = height
        x2 = (width) + _scrollbar_width
        y2 = y1 + _scrollbar_width
        self.body_canvas.create_rectangle(x1, y1, x2, y2, width=0)


In [5]:
class Gui(tk.Tk):
    def __init__(self, import_hook, check, apply_filter):
        super().__init__()

        self.loading_dialog = None
        self.import_hook = import_hook
        self.check = check
        self.apply_filter = apply_filter

        self.start = None
        self.end = None

        self.title("Student Activity Log Analysis")
        self.minsize(1200, 200)
        self.geometry("1200x500")

        # Configure the root window grid
        self.grid_rowconfigure(1, weight=1)  # Middle row expands
        self.grid_columnconfigure(0, weight=1)

        # Top row
        self.top_frame = tk.Frame(self, height=40)
        self.top_frame.grid(row=0, column=0, sticky="nsew")
        self.top_frame.grid_propagate(False)

        # tabs
        notebook = ttk.Notebook(self)
        notebook.grid(row=1, column=0, sticky="nsew")

        # Create frames for tabs
        self.tab_merge_data = ttk.Frame(notebook)
        self.tab_pivot_data_month = ttk.Frame(notebook)
        self.tab_pivot_data_total = ttk.Frame(notebook)
        self.tab_stats_month = ttk.Frame(notebook)
        self.tab_stats_year = ttk.Frame(notebook)
        self.tab_corr = ttk.Frame(notebook)
        self.tab_graphs = ttk.Frame(notebook)

        notebook.add(self.tab_merge_data, text="Merged Data")
        notebook.add(self.tab_pivot_data_month, text="Pivot Data (Monthly)")
        notebook.add(self.tab_stats_month, text="Statistics (Monthly)")
        notebook.add(self.tab_pivot_data_total, text="Pivot Data (Total)")
        notebook.add(self.tab_stats_year, text="Statistics (Total)")
        notebook.add(self.tab_corr, text="Correlation Matrix")
        notebook.add(self.tab_graphs, text="Correlation Graphs")

        self._set_top_bar()
        self.after_idle(self._on_start)

    def _on_start(self):
        if self.check():
            self.apply_filter()
            self.load_csv_button.destroy()
            self.load_csv_button = None

    @staticmethod
    def _clear_frame(frame):
        # Iterate through all widgets in the frame and destroy them
        for widget in frame.winfo_children():
            widget.destroy()

    def show_merge_data(self, header, data):
        if self.load_csv_button:
            self.load_csv_button.destroy()
        self._clear_frame(self.tab_merge_data)
        TableView(self.tab_merge_data, header, data)

    def show_pivot_data_month(self, header, data):
        self._clear_frame(self.tab_pivot_data_month)
        TableView(self.tab_pivot_data_month, header, data)

    def show_pivot_data_total(self, header, data):
        self._clear_frame(self.tab_pivot_data_total)
        TableView(self.tab_pivot_data_total, header, data)

    def show_stats_month(self, header, data):
        self._clear_frame(self.tab_stats_month)
        TableView(self.tab_stats_month, header, data)

    def show_stats_year(self, header, data):
        self._clear_frame(self.tab_stats_year)
        TableView(self.tab_stats_year, header, data)

    def show_corr(self, header, data):
        self._clear_frame(self.tab_corr)
        TableView(self.tab_corr, header, data)

    def show_graphs(self, header, data):
        # self._clear_frame(self.tab_graph)
        # TableViewWithBorders(tab_graph)
        pass

    def _set_top_bar(self):
        tk.Label(self.top_frame, text="Filter by Date:").pack(side=tk.LEFT, padx=(10, 0))
        self.filter_label = tk.Label(self.top_frame)
        self.filter_label.pack(side=tk.LEFT, padx=(10, 0))

        self.filter_button = tk.Button(self.top_frame, text="Apply Date Range", command=self.filter_data)
        self.filter_button.pack(padx=4, pady=2, side=tk.LEFT)
        self._set_filter_text()

        self.load_csv_button = tk.Button(self.top_frame, text="Load CSV Files", command=self.import_files)
        self.load_csv_button.pack(padx=4, pady=2, side=tk.RIGHT)

        ttk.Separator(self.top_frame, orient="vertical").pack(fill="y", padx=10, pady=8, side=tk.LEFT)

    def _set_filter_text(self):
        if self.start and self.end:
            txt = f'{self.start.strftime("%d %b, %Y, %H:%M")} → {self.end.strftime("%d %b, %Y, %H:%M")}'
            self.filter_label.config(text=txt)

    def set_status(self, status: str):
        self.toggle_loading_dialog(False)

        message = f"Message\n\n{status}"
        dialog = tk.Toplevel(self)
        dialog.title("Status Message")
        dialog.geometry("300x100")
        dialog.resizable(False, False)

        dialog.transient(self)
        dialog.grab_set()

        # Add a label with the loading message
        label = tk.Label(dialog, text=message, font=("Arial", 12))
        label.pack(pady=20)

        # Center the loading dialog on the screen
        x = self.winfo_x() + (self.winfo_width() // 2) - 150
        y = self.winfo_y() + (self.winfo_height() // 2) - 50
        dialog.geometry(f"+{x}+{y}")

    def import_files(self):
        files = filedialog.askopenfilenames(
            title="Select Files",
            filetypes=(("CSV Files", "*.csv"), ("All Files", "*.*"))
        )
        if self.import_hook is not None:
            self.import_hook(files)

    def filter_data(self):
        picker = DateTimePickerRange(self, self.start, self.end)
        picker.grab_set()
        self.wait_window(picker)
        if picker.result is not None:
            self.start, self.end = picker.result
        self._set_filter_text()
        self.apply_filter(self.start, self.end)

    def show_table_data(self, headers, data):
        if not isinstance(headers, tuple):
            self.set_status("improper header format")

    def set_dates(self, start_date, end_date):
        self.start = start_date
        self.end = end_date
        self._set_filter_text()

    def toggle_loading_dialog(self, show=False):
        if show:
            message = "Loading and transforming data.\nPlease wait..."
            self.loading_dialog = tk.Toplevel(self)
            self.loading_dialog.title("Loading")
            self.loading_dialog.geometry("300x100")
            self.loading_dialog.resizable(False, False)

            self.loading_dialog.protocol("WM_DELETE_WINDOW", lambda: None)

            # Disable interactions with the main window
            self.loading_dialog.transient(self)
            self.loading_dialog.grab_set()

            # Add a label with the loading message
            label = tk.Label(self.loading_dialog, text=message, font=("Arial", 12))
            label.pack(pady=20)

            # Center the loading dialog on the screen
            x = self.winfo_x() + (self.winfo_width() // 2) - 150
            y = self.winfo_y() + (self.winfo_height() // 2) - 50
            self.loading_dialog.geometry(f"+{x}+{y}")
            # self.update()
            self.update_idletasks()
        else:
            if self.loading_dialog:
                self.loading_dialog.destroy()
                self.loading_dialog = None


In [6]:
def valid_data(user_log_df, activity_log_df):
    intersection = np.intersect1d(user_log_df.columns, activity_log_df.columns)

    user_int = user_log_df[intersection]
    activity_int = activity_log_df[intersection]

    # if the rows, that are common in both data frames do intersect with different data
    # the data is deemed invalid and processing cannot continue
    return (user_int == activity_int).all(axis=1).all()


In [7]:
def prep_df(dataframes):
    if dataframes is None:
        gui.set_status("No data selected")
        return

    user_log_df = next((f for f in dataframes if "Date" in f.columns), None)
    activity_log_df = next((f for f in dataframes if "Action" in f.columns), None)
    component_df = next((f for f in dataframes if "Code" in f.columns), None)

    if user_log_df is None:
        gui.set_status("User Log Data Not Found")
        return
    if activity_log_df is None:
        gui.set_status("Activity Log Data Not Found")
        return
    if component_df is None:
        gui.set_status("Component Data Not Found")
        return

    if not valid_data(user_log_df, activity_log_df):
        gui.set_status("Data is not valid")
        return
    return user_log_df, activity_log_df, component_df

In [8]:
def transformation_remove(user_log_df, activity_log_df, component_df):
    # 1. REMOVE: No outputs should include any data from Component: System, and Folder.
    delete_col_name = '__delete_row'

    # Function to mark rows for deletion based on a condition
    def mark_for_deletion(df, cond):
        df.loc[cond, delete_col_name] = True
        return df

    # Mark rows for deletion
    condition = (lambda df: (df['Component'] == 'System') | (df['Component'] == 'Folder'))

    component_df = mark_for_deletion(component_df, condition(component_df))
    activity_log_df = mark_for_deletion(activity_log_df, condition(activity_log_df))

    # Propagate deletion from activity_log_df to user_log_df
    filtered_indexes = activity_log_df[activity_log_df[delete_col_name] == True].index
    user_log_df.loc[filtered_indexes, delete_col_name] = True

    # Drop marked rows
    def drop_marked_rows(df):
        df.drop(df[df[delete_col_name] == True].index, inplace=True)  # Drop rows in place
        df.drop(columns=[delete_col_name], inplace=True)  # Drop the marker column in place
        df.reset_index(drop=True, inplace=True)  # Reset the index in place

    drop_marked_rows(user_log_df)
    drop_marked_rows(activity_log_df)
    drop_marked_rows(component_df)

    return len(user_log_df) != len(activity_log_df)


In [9]:
def transformation_rename(user_log_df, activity_log_df):
    # 2. RENAME: The column “User Full Name *Anonymized” should be renamed
    # as User_ID both in ACTIVITY_LOG and USER_LOG CSVs.
    col_name_change = {'User Full Name *Anonymized': 'User_ID'}
    activity_log_df.rename(columns=col_name_change, inplace=True)
    user_log_df.rename(columns=col_name_change, inplace=True)

In [10]:
def transformation_merge(user_log_df, activity_log_df):
    # 3. MERGE: Merge the suitable CSVs for analysing user interactions with each component.
    df = pd.concat([user_log_df, activity_log_df.drop('User_ID', axis=1)], axis=1)

    df['Date'] = (pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M') +
                  pd.to_timedelta(df['Time']))

    # Drop the original 'Date' and 'Time' columns if no longer needed
    merge_log_df = df.drop(columns=['Time']).rename(columns={'Date': 'Datetime'})
    return merge_log_df


In [11]:
def transformation_reshape(merge_log_df, col_name, period):
    # 4. RESHAPE: Reshape the data using pivot operation.

    merge_log_df[col_name] = merge_log_df['Datetime'].dt.to_period(period)

    pivot_data = merge_log_df.pivot_table(
        index=['User_ID', col_name],
        columns='Component',
        values='Action',
        aggfunc='count',
        fill_value=0
    )

    pivot_data.columns = [col for col in pivot_data.columns]

    return pivot_data.reset_index()


In [12]:
def transformation_count(pivot_data, col_name):
    # 5. COUNT: The interactions for each user with the Component for each month.
    pivot_data['Total Interaction'] = pivot_data.loc[:, ~pivot_data.columns.isin(['User_ID', col_name])].sum(axis=1)


In [13]:
def transformation_output_statistics(pivot_data, period):
    #     OUTPUT STATISTICS: Produce the mean, mode and median for the components: Quiz, Lecture, Assignment, Attendance, and Survey.
    # - For each month
    # - For the entire 13-week academic semester
    selected_components = ['User_ID', period, 'Quiz', 'Lecture', 'Assignment', 'Attendence', 'Survey']
    columns_in_df = [c for c in selected_components if c in pivot_data.columns]
    raw_data = pivot_data[columns_in_df]
    monthly_stats = raw_data[['User_ID', period]].copy()
    data_df = raw_data.loc[:, ~raw_data.columns.isin(['User_ID', period])]
    monthly_stats['Mean'] = data_df.fillna(0).mean(axis=1)
    monthly_stats['Median'] = data_df.median(axis=1)
    mode = data_df.mode(axis=1)
    if not mode.empty:
        monthly_stats['Mode'] = mode.iloc[:, 0]
    else:
        monthly_stats['Mode'] = pd.NA
    return monthly_stats


In [14]:
def transformation_output_correlation(pivot_data):
    correlation_components = ['Assignment', 'Quiz', 'Lecture', 'Book', 'Project', 'Course']
    correlation_data = pivot_data[[c for c in correlation_components if c in pivot_data.columns]]
    corr_matrix = correlation_data.corr()
    corr_matrix_with_labels = corr_matrix.reset_index()

    # Rename the new first column
    return corr_matrix_with_labels.rename(columns={"index": "Components"})

In [15]:
headerDimensions = {
    'Datetime': 10,
    'User_ID': 5,
    'Component': 8,
    'Action': 6,
    'Target': 10,
    'Month': 8,
    'Year': 6,
}


def extract_data(df: pd.DataFrame):
    headers = []
    for col in df.columns:
        dim = headerDimensions[col] if col in headerDimensions else 8
        headers.append((col, dim))
    return headers, df.values.tolist()

In [16]:
from pathlib import Path
from datetime import datetime


class DbConn:
    def __init__(self):
        fp = Path("data")
        if not fp.exists():
            fp.mkdir()

        self._db_name = "data/db.sqlite"
        self._kv_store_table = "kv_store"
        self._create_kv_store()

    def _connect(self):
        return sqlite3.connect(self._db_name)

    def save(self, df, table_name, if_exists='replace', index=False, chunksize=1000):
        with self._connect() as conn:
            df.to_sql(table_name, conn, if_exists=if_exists, index=index, chunksize=chunksize)
            cursor = conn.cursor()
            cursor.execute(f'create index idx_datetime on {table_name} (Datetime)')
            conn.commit()

    def data_exists(self, table_name):
        with self._connect() as conn:
            cursor = conn.execute("select name from sqlite_master where type='table' and name=?", (table_name,))
            if cursor.fetchone() is not None:
                cursor = conn.execute(f"select count(*) from {table_name}")
                return cursor.fetchone()[0]
            else:
                return False

    def load(self, table_name, start, end):
        suffix = ''
        params = None
        if start and end:
            suffix = f' where Datetime between ? and ?'
            params = (start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S'))
        with self._connect() as conn:
            df = pd.read_sql(f'select * from {table_name}{suffix}', conn, params=params)
        return df

    def _create_kv_store(self):
        """Create the key-value table if it doesn't exist."""
        with self._connect() as conn:
            conn.execute(f"create table if not exists {self._kv_store_table} (key text primary key,value text)")

    def set_datetime_val(self, key, value):
        """Insert or update a key-value pair."""
        value = value.strftime('%Y-%m-%d %H:%M:%S')

        with self._connect() as conn:
            conn.execute(f"""
                insert into {self._kv_store_table} (key, value)
                values (?, ?)
                on conflict(key) do update set value = excluded.value
            """, (key, value))

    def get_datetime_val(self, key):
        """Retrieve a value by key."""
        with self._connect() as conn:
            cursor = conn.execute(f"""
                select value from {self._kv_store_table} where key = ?
            """, (key,))
            result = cursor.fetchone()
            return datetime.strptime(result[0]) if result else None


In [17]:

def import_hook(files):
    dataframes = []
    try:
        for file in files:
            dataframes.append(pd.read_csv(file))
    except Exception as e:
        gui.set_status("File Read error: " + str(e))
        return

    gui.toggle_loading_dialog(show=True)

    user_log_df, activity_log_df, components_df = prep_df(dataframes)

    if not transformation_remove(user_log_df, activity_log_df, components_df):
        gui.set_status("Cleaning resulted in incompatible data types")

    transformation_rename(user_log_df, activity_log_df)

    merged_df: pd.DataFrame = transformation_merge(user_log_df, activity_log_df)

    db = DbConn()

    start_date = merged_df["Datetime"].min()
    end_date = merged_df["Datetime"].max()

    db.save(merged_df, table_name="merged_data")

    db.set_datetime_val("start_date", start_date)
    db.set_datetime_val("end_date", start_date)

    apply_transformations(start_date, end_date)


def check():
    return DbConn().data_exists("merged_data")


def apply_transformations(start_date=None, end_date=None):
    db = DbConn()

    if start_date:
        db.set_datetime_val("start_date", start_date)
    else:
        start_date = db.get_datetime_val("start_date")

    if end_date:
        db.set_datetime_val("end_date", start_date)
    else:
        end_date = db.get_datetime_val("end_date")

    merged_df = db.load("merged_data", start_date, end_date)
    merged_df['Datetime'] = pd.to_datetime(merged_df['Datetime'], format='%Y-%m-%d %H:%M:%S')

    if start_date is None:
        start_date = merged_df["Datetime"].min()

    if end_date is None:
        end_date = merged_df["Datetime"].max()

    gui.set_dates(start_date, end_date)

    header, data = extract_data(merged_df)
    gui.show_merge_data(header, data)

    # monthly statistics
    month_pivot_df = transformation_reshape(merged_df.copy(), "Month", period='M')

    header, data = extract_data(month_pivot_df)
    gui.show_pivot_data_month(header, data)

    transformation_count(month_pivot_df, "Month")
    monthly_stats = transformation_output_statistics(month_pivot_df, "Month")
    header, data = extract_data(monthly_stats)
    gui.show_stats_month(header, data)

    # # total statistics
    year_pivot_df = transformation_reshape(merged_df.copy(), "Year", period='Y')
    header, data = extract_data(year_pivot_df)
    gui.show_pivot_data_total(header, data)
    year_stats = transformation_output_statistics(year_pivot_df, "Year")
    header, data = extract_data(year_stats)
    gui.show_stats_year(header, data)

    corr_metrix = transformation_output_correlation(year_pivot_df)
    header, data = extract_data(corr_metrix)
    gui.show_corr(header, data)

    gui.toggle_loading_dialog(show=False)




In [ ]:
gui = Gui(import_hook, check, apply_transformations)
gui.mainloop()

todo
- use searbon for plot - https://seaborn.pydata.org/examples/horizontal_boxplot.html
- save start and end date in sqlite db
- test date limits